In [1]:
import numpy as np
import json
import os
import sys
import glob
import typing as T
import shutil
import cv2
import argparse
import open3d as o3d
from cam_settings import cam_series,camera_set
from read_color_raw422_multi import generate_png_from_raw
id2cam = ["1246","2543","0385","1973","1634","1040","4320","0879","1362","1228","0028","0244","2129","2448","1516","1169","0103","1265","1116","1753","8540","1285","1100","1318","1705"]
cam2id = {}
for i, cam in enumerate(id2cam):
    cam2id[cam] = i
# calculate fov from intrinsic matrix
def calculate_fov(intrinsic_matrix, image_width, image_height):
    # Extract focal lengths from the intrinsic matrix
    f_x = intrinsic_matrix[0, 0]
    f_y = intrinsic_matrix[1, 1]
    
    # Calculate the horizontal FOV
    fov_x = 2 * np.arctan(image_width / (2 * f_x)) * (180 / np.pi)
    
    # Calculate the vertical FOV
    fov_y = 2 * np.arctan(image_height / (2 * f_y)) * (180 / np.pi)
    
    return fov_x, fov_y

def generate_camera_json(
        meta_path='./meta_data/043422251246-MODEL.json',
        transform_path: T.Optional[str] = None):
    
    if transform_path is None:
        extrinsics = np.eye(4)
    else:
        extrinsics = np.loadtxt(transform_path, delimiter=' ', dtype=np.float32)

    with open(meta_path, 'r') as f:
        meta = json.load(f)
    intrinsics = meta['color_intrinsics']
    intrinsic_matrix = np.array([[intrinsics['fx'], 0, intrinsics['ppx']],
                                [0, intrinsics['fy'], intrinsics['ppy']],
                                [0, 0, 1]])
    intrinsic_matrix = intrinsic_matrix.astype(np.float32)
    H_c2w = extrinsics

    fov_x, fov_y = calculate_fov(intrinsic_matrix, intrinsics['width'], intrinsics['height'])
    return H_c2w, intrinsic_matrix, fov_x, fov_y



In [2]:
frame_id = 50
current_frame = str(frame_id).zfill(7)
root_path = 'cali1'
output_path = 'cali1/billy_9-23'
pcd_path = 'combined_pcd_1246.ply'

raw_path = os.path.join(root_path, 'raw_data', current_frame)
meta_path = os.path.join(root_path, 'meta_data')
trans_path = os.path.join(root_path, 'output_data') 

if os.path.exists(output_path):
    shutil.rmtree(output_path)
os.mkdir(output_path)

train_output_path = os.path.join(output_path,'train')
test_output_path = os.path.join(output_path,'test')
os.mkdir(train_output_path)
pcd = o3d.io.read_point_cloud(pcd_path)
o3d.io.write_point_cloud(os.path.join(output_path,'pcd_0.ply'), pcd)

# read all camera info from meta_data
target_cam = '1246'
source_cam = [cam_id for cam_id in list(cam_series.keys()) if cam_id not in camera_set[8]]

info_dict = {}
for cam_id in source_cam:
    serie_id = cam_series[cam_id]
    cur_meta_path = os.path.join(meta_path, f'{serie_id}-MODEL.json')
    cur_trans_path = os.path.join(trans_path, f'{cam_id}_to_{target_cam}_H_fine.txt')
    if cam_id == target_cam:
        cur_trans_path = None
    H_c2w, intrinsic_matrix, fov_x, fov_y = generate_camera_json(cur_meta_path, cur_trans_path)
    info_dict[cam_id] = dict(
        H_c2w = H_c2w,
        intrinsic_matrix = intrinsic_matrix,
    )

# progressively add the nearest camera
added_cams = [target_cam]
added_tvecs = [np.zeros(3)]
free_cams = [cam_id for cam_id in source_cam if cam_id not in added_cams]
while len(free_cams) > 0:
    min_dist = float('inf')
    min_cam = None
    for cam_id in free_cams:
        curr_tvec = info_dict[cam_id]['H_c2w'][:3, 3]
        dist_to_added = added_tvecs - curr_tvec
        dist_to_added = np.linalg.norm(dist_to_added, axis=1)
        cur_to_added = np.min(dist_to_added)
        if  cur_to_added< min_dist:
            min_dist = cur_to_added
            min_cam = cam_id
    added_cams.append(min_cam)
    added_tvecs.append(info_dict[min_cam]['H_c2w'][:3, 3])
    free_cams.remove(min_cam)
    print(f'added {min_cam} with distance {min_dist}')



added 2543 with distance 0.3717506015414412
added 0385 with distance 0.3423120835902946
added 1973 with distance 0.6653784450675337
added 1634 with distance 0.9683077224034023
added 1040 with distance 0.6628199952650149
added 4320 with distance 0.6774573916037263
added 0879 with distance 1.014700420040605
added 1362 with distance 0.6453985565640087
added 1228 with distance 0.6660300034236019
added 0028 with distance 1.1169620065598111
added 0244 with distance 0.6412533686038402
added 2129 with distance 0.6737315613012448
added 2448 with distance 0.9916180719131731
added 1516 with distance 0.6547318901243162
added 1169 with distance 0.6690283909928747
added 0103 with distance 0.9979305270058818
added 1265 with distance 0.616944177078708
added 1116 with distance 0.672997328820711
added 1753 with distance 0.9841905265681862
added 8540 with distance 0.6848257509372315
added 1285 with distance 0.6922242865933831
added 1100 with distance 1.0969596119450287
added 1318 with distance 0.66853015

In [5]:
add_cam_str = "\",\"".join(added_cams)
print(add_cam_str)

1246","2543","0385","1973","1634","1040","4320","0879","1362","1228","0028","0244","2129","2448","1516","1169","0103","1265","1116","1753","8540","1285","1100","1318","1705


In [3]:
H_c2w_list = []
intrinsic_matrix_list = []
for cam_id in added_cams:
    H_c2w_list.append(info_dict[cam_id]['H_c2w'])
    intrinsic_matrix_list.append(info_dict[cam_id]['intrinsic_matrix'])

camera_dict = {
        "H_c2w": [
            [
                x.tolist() for x in H_c2w_list
            ]
        ],
        "intrinsic": [
            [
                x.tolist() for x in intrinsic_matrix_list
            ]
        ],
        "width_px": 1280,
        "height_px": 800,
        "fov_x_deg": fov_y,
        "fov_y_deg": fov_y,
    }
train_output_path = os.path.join(output_path,'train')
test_output_path = os.path.join(output_path,'test')
# dump
camera_json_path = os.path.join(train_output_path, 'camera.json')
print(camera_json_path)
with open(camera_json_path, 'w') as f:
    json.dump(camera_dict, f, indent=4)

# create ground truth views
def process_img(img_path):
    img = cv2.imread(img_path)
    # pad to 1280x1280
    pad_h = (1280 - img.shape[0]) // 2
    pad_w = (1280 - img.shape[1]) // 2
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant')
    return padded

train_input_path = os.path.join(train_output_path, 'input')
os.mkdir(train_input_path)
for i, cam_id in enumerate(added_cams):
    serie_id = cam_series[cam_id]
    rgb_id = str(i).zfill(2)
    color_view = os.path.join(train_input_path, f'rgb_{rgb_id}.png')
    generate_png_from_raw(os.path.join(raw_path, f'{serie_id}-COLOR.{current_frame}.raw'),\
        800,1280, color_view)

shutil.copytree(train_output_path, test_output_path)

cali1/billy_9-23/train/camera.json


'cali1/billy_9-23/test'